In [ ]:
from typing import Tuple

import numpy as np
from scipy import stats
from scipy.optimize import fsolve

In [ ]:
N = 6
S = 8
I1 = 5
I2 = 4

alpha_1 = 0.1
alpha_2 = 0.01

tau = 1 / (1 + I1)
p1 = (alpha_1 ** (1 - tau)) * (alpha_2 ** tau)
p2 = 10 * np.sqrt(S) * p1

dist_family = {
    1: {"theta": (0, 1), "dist": stats.t},
    2: {"theta": (S, N), "dist": stats.gamma},
    3: {"theta": (N, S + 2), "dist": stats.pareto},
    4: {"theta": (S, S + N), "dist": stats.beta},
    5: {"theta": (-N, S + 4), "dist": stats.uniform},
}


def compute_theta_1(distribution: stats.rv_continuous, theta_0: Tuple[float, float]) -> Tuple[float, float]:
    mu0 = distribution.mean(*theta_0)
    v0 = distribution.var(*theta_0)

    def equations(params: Tuple[float, float]) -> Tuple[float, float]:
        scale, shape = params
        return (distribution.mean(scale, shape) - mu0,
                distribution.var(scale, shape) - N * v0)

    _theta_1 = fsolve(equations, np.array(theta_0))
    return _theta_1[0].item(), _theta_1[1].item()


def compute_theta_2(distribution: stats.rv_continuous, theta_0: Tuple[float, float]) -> Tuple[float, float]:
    mu0 = distribution.mean(*theta_0)
    v0 = distribution.var(*theta_0)

    def equations(params):
        scale, shape = params
        return (stats.pareto.mean(scale, shape) - (mu0 + 2 * np.sqrt(v0)),
                stats.pareto.var(scale, shape) * S - v0)

    _theta_2 = fsolve(equations, np.array(theta_0))
    return _theta_2[0].item(), _theta_2[1].item()


print(f"""
N = {N}
S = {S}
I1 = {I1}
I2 = {I2}
alpha_1 = {alpha_1}
alpha_2 = {alpha_2}
tau = {tau}
p1 = {p1}
p2 = {p2}
""")

In [ ]:
family = dist_family[5]
family

In [ ]:
theta_1 = compute_theta_1(distribution=family["dist"], theta_0=family["theta"])
theta_1

In [ ]:
theta_2 = compute_theta_2(distribution=family["dist"], theta_0=family["theta"])
theta_2